In [1]:
# 1/3

# Discontinuous Galerkin 

from mshr import *
from dolfin import *
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import scipy.sparse as sparse
import numpy as np

# # -----------
# # Libraries for stability analysis
# from __future__ import print_function
# from __future__ import division
# from IPython.display import clear_output
# from scipy.optimize import brenth
# from scipy.optimize import fsolve
# from scipy.linalg import eigvals
# from scipy.linalg import eig
# from scipy.sparse.linalg import eigs
# from scipy.interpolate import interp1d
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
# from matplotlib import rc
# import matplotlib
# import scipy.signal
# from math import pi
# import time
# import math
# %matplotlib inline
# -----------

# OPERATORS
zcross = lambda i: as_vector((-i[1],i[0]))

# BASIN & MESH
length = 5.0
width  = 5.0
resolution = 3
geometry  = Rectangle(Point(0.0, 0.0), Point(width, length))
mesh = generate_mesh(geometry, resolution)

NameError: name 'DG' is not defined

In [2]:
plot(mesh);

NameError: name 'plot' is not defined

In [3]:
# 2/3

# FUNCTION & VECTOR SPACES
DG = VectorElement("DG", mesh.ufl_cell(), 1)
CG = FiniteElement("CG", mesh.ufl_cell(), 2)
G = FunctionSpace(mesh, MixedElement((DG, CG)))

# TRIAL/TEST FUNCTIONS
(u, eta) = TrialFunctions(G)
(v, lmbda) = TestFunctions(G)

# EIGENVALUE SOLs
eigenvalues_real = []
eigenvalues_imag = []

# VARIABLES
F    = Constant("1.0")
Ro   = Constant("1.0")
Fcor = Constant("0.0")
#Fcor = Constant("1.0")

# SET-UP EQUATION
a = inner(v, Fcor*zcross(u))*dx \
    + inner(v, grad(eta))*dx \
    - inner(u, grad(lmbda))*dx

m = Ro*inner(v, u)*dx + Ro*F*lmbda*eta*dx

A = PETScMatrix()
assemble(a, tensor = A)
M = PETScMatrix()
assemble(m, tensor = M)

A_mat = as_backend_type(A).mat()

A_sparray = csr_matrix(A_mat.getValuesCSR()[::-1], shape = A_mat.size)

# print(A_sparray)
plt.spy(A_sparray)
plt.rcParams ['figure.figsize'] = [12, 8]
plt.grid (True, which = "both")
plt.xlabel("y dim")
plt.ylabel("x dim")

plt.show()

NameError: name 'VectorElement' is not defined

In [4]:
# 2/3

# Create eigensolver
# eig_num    = 100
eigensolver = SLEPcEigenSolver(A, M)

# # -----
# # Number of eigenvalues to be extracted

# # Scipy Eigen package
# eig_vals, eig_vecs = sc.sparse.linalg.eigsh(sc_Kt_sld, k=eign_num, which='SA', mode='buckling')
# # -----

# eigensolver.parameters["spectrum"] = "target imaginary"
# eigensolver.parameters["spectral_transform"] = "shift-and-invert"
# eigensolver.parameters["spectral_shift"] = 1.0

# Compute all eigenvalues of A x = \lambda x
print("Computing eigenvalues. This can take a minute.")
eigensolver.solve()
# eigensolver.solve(n)

listreal = []
listimag = []

# Computing the eigenvalues
for i in range(eigensolver.get_number_converged()):
    
    print ("i = ", i)
    
    r, c, rx, cx = eigensolver.get_eigenpair(i)
    
    listreal.append (r)
    print ("real      =", r)
    
    listimag.append (c)
    print ("imaginary =", c)

# Extract largest (first) eigenpair
r, c, rx, cx = eigensolver.get_eigenpair(0)
print("")

print("Largest REAL eigenvalue: ", r)
print("Largest IMAGINARY eigenvalue: ", c)


NameError: name 'SLEPcEigenSolver' is not defined

In [ ]:
# 3/3

import matplotlib

eigenvalues_real.append(r)
eigenvalues_imag.append(c)

# DEFN FUNCTION SPACES
eigenmodes_real = Function(G, rx)
eigenmodes_imag = Function(G, cx)

# ur, etar = eigenmodes_real.split()
# ui, etai = eigenmodes_imag.split()

# # LIST EIGENVALUES
# print ("Real Eigenfrequencies:"), eigenvalues_real
# print ("Imaginary Eigenfrequencies:"), eigenvalues_imag

# SAVE LAST REAL
File("eigenvalues_test/eigenmodes_real.pvd") << eigenmodes_real

# SAVE LAST IMAGINARY
File("eigenvalues_test/.pvd") << eigenmodes_imag

# TUTORIALS
# https://fenicsproject.org/qa/13796/formulating-an-eigenvalue-problem/
# https://fenicsproject.org/qa/8680/solving-generalized-eigenvalues-finite-element-stiff-matrix/

# Characteristics for plots
liststyles       = ["--", "-", "-.", "."]
listcolor        = ["k", "g", "b", "r", "none"]
listmarkers      = ["s", "o", "^", ">", "<", "p"]

# Plot eigenspectra
fig, ax = plt.subplots ()
area = 50
            
ax.scatter (listreal, listimag, s = area, marker = listmarkers [0], color = listcolor [4], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5, label = 'mesh resolution = ' + str(resolution))
ax.set_xscale ('symlog', linthreshx = 1e-17)
# ax.set_yscale ('symlog')

ax.xaxis.set_minor_locator(MultipleLocator(10000))

plt.rcParams ['figure.figsize'] = [12, 8]
leg1 = ax.legend (loc = 'upper right', frameon = True, fontsize = 14);
plt.grid (True, which = "both")
plt.xlabel ('Re $[\omega]$ [1/s]', fontsize = 18)
plt.ylabel ('Im $[\omega]$ [1/s]', fontsize = 16)
        
matplotlib.rc ('xtick', labelsize = 18)     
matplotlib.rc ('ytick', labelsize = 18)

plt.ylim (( - 10.5 , 10.5))
plt.xlim (( - 1e-11, 1e-11))

plt.show ()